In [ ]:
# this is for a strange bug with compute instances 
import os

os.system('sudo mkdir temp')
os.system('sudo cp /etc/nginx/nginx.conf temp/temp.conf') # stupid

nginx = ''

with open('temp/temp.conf') as f:
    for line in f.readlines():
        if 'websocket/|/ws/' in line:
            nginx += line.replace('websocket/|/ws/', 'websocket/|/ws')
        else:
            nginx += line

with open('temp/temp2.conf', 'w') as f:
    f.write(nginx)
    
os.system('sudo mv temp/temp2.conf /etc/nginx/nginx.conf')
os.system('sudo service nginx restart')
# os.system('rm setup/temp.conf')
os.system('sudo rm -rf temp')

In [9]:
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment
from azureml.core.runconfig import MpiConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
from azureml.core.runconfig import MpiConfiguration


In [2]:
interactive_auth = InteractiveLoginAuthentication(
        tenant_id='72f988bf-86f1-41af-91ab-2d7cd011db47'
    )

subscription_id = '6560575d-fa06-4e7d-95fb-f962e74efd7a'
resource_group = 'azure-sandbox'
workspace_name = 'todrabas_UK_STH'
location = 'uksouth'

ws = Workspace(
      workspace_name=workspace_name
    , subscription_id=subscription_id
    , resource_group=resource_group
    , auth=interactive_auth
)

In [3]:
# ws = Workspace.from_config()
# from dask_cloudprovider import AzureMLCluster
### name
name = 'todrabas'             # REPLACE

### vnet settings
vnet_rg = ws.resource_group  # replace if needed
vnet_name = 'todrabas_UK_STH_VN'     # replace if needed
subnet_name = 'default'          # replace if needed

### azure ml names
ct_name = f'{name}-gpussh'
exp_name = f'{name}-dask-demo'

### trust but verify
verify = f'''
Name: {name}

vNET RG: {vnet_rg}
vNET name: {vnet_name}
vNET subnet name: {subnet_name}

Compute target: {ct_name}
Experiment name: {exp_name}
'''

print(verify)

# vm_name = list(AzureMLConfigurations.azure_gpu_vms.keys())[6]
# gpus_per_node = AzureMLConfigurations.azure_gpu_vms[vm_name]

if ct_name not in ws.compute_targets:
    # create config for Azure ML cluster
    # change properties as needed
    config = AmlCompute.provisioning_configuration(
          vm_size=vm_name
        , min_nodes=0
        , max_nodes=4
        , vnet_resourcegroup_name=vnet_rg
        , vnet_name=vnet_name
        , subnet_name=subnet_name
        , idle_seconds_before_scaledown=120
        , 
    )
    ct = ComputeTarget.create(ws, ct_name, config)
    ct.wait_for_completion(show_output=True)
else:
    ct = ws.compute_targets[ct_name]

## specify the data and code stores

codefileshare = 'codefileshare'
datafileshare = 'datafiles'
    
environment_name='todrabas_GPU_ENV'
env = ws.environments[environment_name]


Name: todrabas

vNET RG: azure-sandbox
vNET name: todrabas_UK_STH_VN
vNET subnet name: default

Compute target: todrabas-gpussh
Experiment name: todrabas-dask-demo



In [4]:
from azureml.data.data_reference import DataReference

codefileshare_store = Datastore(ws, codefileshare)
datafileshare_store = Datastore(ws, datafileshare)

codefileshare_data = DataReference(
       datastore=codefileshare_store,
       data_reference_name="codes",
       path_on_datastore=".")

datafileshare_data = DataReference(
       datastore=datafileshare_store,
       data_reference_name="data",
       path_on_datastore=".")

In [64]:
scheduler_params = {
    '--scheduler': '10.7.0.5:8786'
    , '--datastore_name': 'datafiles'
    , '--path_to_data': '../../../datafiles/nyctaxi/2015/*'
    , '--path_to_model': '../../codes/GTC/SJ_2020/workshop/saved_models/trained_model_xgboost.mdl'
    , '--path_to_output': '../../codes/GTC/SJ_2020/workshop/saved_models/predictions/'
    , '--file_index_start': 8
}

In [67]:
exp = Experiment(ws, 'model_inference')
estimator = Estimator(
    'inference'
    , compute_target=ct
    , entry_script='NYCTaxi_Dask_XGBoost_Inference.py'
    , environment_definition=env
    , script_params=scheduler_params
    , node_count=1 ### start only scheduler
    , distributed_training=MpiConfiguration()
    , use_docker=True
    , inputs=[codefileshare_data, datafileshare_data]
)

run = exp.submit(estimator)